In [1]:
import os
import shutil
import glob
import numpy as np
import pandas as pd



In [ ]:
from torchvision.datasets import DatasetFolder
from PIL import Image
from torchvision import transforms
import torch
import pandas as pd

class GlucoseDataset(DatasetFolder):
    def __init__(self, img_dir, dataframe, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        glucose_level = self.dataframe.iloc[idx, 1]
        image = Image.open(img_path).convert('RGB')  # Convert grayscale images to RGB
        if self.transform:
            image = self.transform(image)
        return image, glucose_level

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}



In [ ]:

data_dir = "/glucose-prediction/RP_Images/"


train_df = pd.read_csv('/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/Mappings/train_mappings.csv')
val_df = pd.read_csv('/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/Mappings/val_mappings.csv')


train_dataset = GlucoseDataset(data_dir + 'train', train_df, transform=data_transforms['train'])
val_dataset = GlucoseDataset(data_dir + 'val', val_df, transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)






In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import StepLR
from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights

best_model_path = 'best_model_accuracy.pth'

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Adjust the model's fully connected layer for regression
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Loss functions
def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

def mape(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / (y_true + 1e-8)))

# custom 
def combined_loss(y_true, y_pred):
    return rmse(y_true, y_pred) + mape(y_true, y_pred)

# Added function to compute accuracy from MAPE
def accuracy_from_mape(mape_val):
    return 100 - (mape_val*100)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
num_epochs = 200

best_train_rmse, best_val_rmse = float('inf'), float('inf')
best_train_mape, best_val_mape = float('inf'), float('inf')
best_train_loss, best_val_loss = float('inf'), float('inf')
best_train_accuracy, best_val_accuracy = 0.0, 0.0

# training loop
for epoch in range(num_epochs):
    model.train()
    total_loss, total_rmse, total_mape = 0.0, 0.0, 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = combined_loss(labels, outputs)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        total_rmse += rmse(labels, outputs).item() * images.size(0)
        total_mape += mape(labels, outputs).item() * images.size(0)

    avg_train_loss = total_loss / len(train_dataset)
    avg_train_rmse = total_rmse / len(train_dataset)
    avg_train_mape = total_mape / len(train_dataset)
    train_accuracy = accuracy_from_mape(avg_train_mape)

    if avg_train_loss < best_train_loss:
        best_train_loss = avg_train_loss
    if avg_train_rmse < best_train_rmse:
        best_train_rmse = avg_train_rmse
    if avg_train_mape < best_train_mape:
        best_train_mape = avg_train_mape
    if train_accuracy > best_train_accuracy:
        best_train_accuracy = train_accuracy
   
    


    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print("-" * 10)
    print(f'Train Loss: {avg_train_loss:.4f}, RMSE: {avg_train_rmse:.4f}, MAPE: {avg_train_mape:.4f}, Accuracy: {train_accuracy:.2f}%')

    model.eval()
    total_val_loss, total_val_rmse, total_val_mape = 0.0, 0.0, 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            total_val_loss += combined_loss(labels, outputs).item() * images.size(0)
            total_val_rmse += rmse(labels, outputs).item() * images.size(0)
            total_val_mape += mape(labels, outputs).item() * images.size(0)

        avg_val_loss = total_val_loss / len(val_dataset)
        avg_val_rmse = total_val_rmse / len(val_dataset)
        avg_val_mape = total_val_mape / len(val_dataset)
        val_accuracy = accuracy_from_mape(avg_val_mape)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
        if avg_val_rmse < best_val_rmse:
            best_val_rmse = avg_val_rmse
        if avg_val_mape < best_val_mape:
            best_val_mape = avg_val_mape
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), best_model_path)


    print(f'Val Loss: {avg_val_loss:.4f}, RMSE: {avg_val_rmse:.4f}, MAPE: {avg_val_mape:.4f}, Accuracy: {val_accuracy:.2f}%')
    scheduler.step()

print(f'Best Train Loss: {best_train_loss:.4f}, Best Train RMSE: {best_train_rmse:.4f}, Best Train MAPE: {best_train_mape:.4f}, Best Train Accuracy: {best_train_accuracy:.2f}%')
print(f'Best Val Loss: {best_val_loss:.4f}, Best Val RMSE: {best_val_rmse:.4f}, Best Val MAPE: {best_val_mape:.4f}, Best Val Accuracy: {best_val_accuracy:.2f}%')



Epoch [1/200]
----------
Train Loss: 47.0750, RMSE: 46.7031, MAPE: 0.3719, Accuracy: 62.81%
Val Loss: 20.5694, RMSE: 20.4095, MAPE: 0.1599, Accuracy: 84.01%
Epoch [2/200]
----------
Train Loss: 21.8673, RMSE: 21.7150, MAPE: 0.1523, Accuracy: 84.77%
Val Loss: 18.4934, RMSE: 18.3687, MAPE: 0.1247, Accuracy: 87.53%
Epoch [3/200]
----------
Train Loss: 21.7432, RMSE: 21.5923, MAPE: 0.1509, Accuracy: 84.91%
Val Loss: 18.3069, RMSE: 18.1798, MAPE: 0.1271, Accuracy: 87.29%
Epoch [4/200]
----------
Train Loss: 21.8177, RMSE: 21.6677, MAPE: 0.1500, Accuracy: 85.00%
Val Loss: 18.3219, RMSE: 18.1916, MAPE: 0.1304, Accuracy: 86.96%
Epoch [5/200]
----------
Train Loss: 21.7189, RMSE: 21.5692, MAPE: 0.1496, Accuracy: 85.04%
Val Loss: 18.5512, RMSE: 18.4149, MAPE: 0.1362, Accuracy: 86.38%
Epoch [6/200]
----------
Train Loss: 21.6701, RMSE: 21.5192, MAPE: 0.1508, Accuracy: 84.92%
Val Loss: 18.2922, RMSE: 18.1641, MAPE: 0.1282, Accuracy: 87.18%
Epoch [7/200]
----------
Train Loss: 21.9364, RMSE: 21.786